In [1]:
# Install required libraries
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 96.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling 

In [2]:
import cv2
import numpy as np
from ultralytics import YOLO
from google.colab import drive


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Load YOLO model
model = YOLO('yolov8n.pt')

# Load the video from Google Drive
cap = cv2.VideoCapture('/content/drive/MyDrive/Traking_video/input_video1.mp4')
if not cap.isOpened():
    print("Error: Could not open video file 'input_video.mp4' in Google Drive")
    exit()

100%|██████████| 6.25M/6.25M [00:00<00:00, 117MB/s]


In [5]:
# Get video properties
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
fps = int(cap.get(5))
print(f"Video properties - Width: {frame_width}, Height: {frame_height}, FPS: {fps}")

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('/content/drive/MyDrive/output_video.mp4', fourcc, fps, (frame_width, frame_height))
if not out.isOpened():
    print("Error: Could not open output video writer")
    exit()

Video properties - Width: 1280, Height: 720, FPS: 30


In [6]:
# Initialize lists to store tracked points for the first person
tracked_points = []
selected_person_center = None

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Failed to read frame")
        break
    print("Processing frame")

    # Perform object detection
    results = model(frame)

    # Process detections
    person_detected = False
    closest_center = None
    closest_distance = float('inf')

    for result in results:
        boxes = result.boxes.xyxy.cpu().numpy()
        classes = result.boxes.cls.cpu().numpy()
        confidences = result.boxes.conf.cpu().numpy()

        for box, cls, conf in zip(boxes, classes, confidences):
            if cls == 0 and conf > 0.3:
                print(f"Person detected with confidence {conf}")
                x1, y1, x2, y2 = map(int, box)
                center = ((x1 + x2) // 2, (y2 + y1) // 2)
                person_detected = True

                if selected_person_center is None:
                    selected_person_center = center
                    tracked_points.append(center)
                    print(f"Locked onto first person at {center}")
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.putText(frame, 'Selected Person', (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                else:
                    distance = np.sqrt((center[0] - tracked_points[-1][0])**2 + (center[1] - tracked_points[-1][1])**2)
                    if distance < closest_distance:
                        closest_distance = distance
                        closest_center = center

    if closest_center and closest_distance < 100:
        tracked_points.append(closest_center)
        print(f"Added point: {closest_center}")

    # Draw tracked route in thinner red lines if a person was detected
    if person_detected and len(tracked_points) > 1:
        for i in range(1, len(tracked_points)):
            if tracked_points[i-1] and tracked_points[i]:
                cv2.line(frame, tracked_points[i-1], tracked_points[i], (0, 0, 255), 2)

    # Write the frame to output video
    out.write(frame)

# Release resources
cap.release()
out.release()

print("Video processing complete. Check '/content/drive/MyDrive/output_video.mp4'.")

Streaming output truncated to the last 5000 lines.
Person detected with confidence 0.8581904172897339
Person detected with confidence 0.8213813900947571
Person detected with confidence 0.4420154392719269
Added point: (534, 306)
Processing frame

0: 384x640 4 persons, 6 chairs, 10.4ms
Speed: 2.4ms preprocess, 10.4ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)
Person detected with confidence 0.8761029839515686
Person detected with confidence 0.7406489253044128
Person detected with confidence 0.7208872437477112
Person detected with confidence 0.37459632754325867
Added point: (540, 311)
Processing frame

0: 384x640 4 persons, 6 chairs, 8.8ms
Speed: 2.8ms preprocess, 8.8ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)
Person detected with confidence 0.8760756850242615
Person detected with confidence 0.7408891320228577
Person detected with confidence 0.7147921919822693
Person detected with confidence 0.3589339554309845
Added point: (539, 311)
Processin